<a href="https://colab.research.google.com/github/karaage0703/karaage-ai-book/blob/master/ch02/02_karaage_ai_book_image_classification_on_camera_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demonstration of image classification on camera
Image classification is performed on live images being captured by the camera.


## Import the necessary libraries

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import time

## Loading pre-trained AI model


In [ ]:
!wget "https://raw.githubusercontent.com/fu11ji26/janken-classification-lesson/master/efficientnet_model.h5" -O "efficientnet_model.h5"
import tensorflow_hub as hub
keras_model = tf.keras.models.load_model('efficientnet_model.h5', custom_objects={'KerasLayer':hub.KerasLayer})
keras_model.summary()

Loading labels (= gu, choki, pa).

In [ ]:
!wget "https://raw.githubusercontent.com/fu11ji26/janken-classification-lesson/master/labels.txt" -O "labels.txt"
labels = []
with open('labels.txt','r') as f:
  for line in f:
    labels.append(line.rstrip())
print(labels)

NUM_CLASSES = len(labels)

## Demonstration of image classification

Download the Javascript code to use the webcam.

In [ ]:
!wget https://raw.githubusercontent.com/fu11ji26/janken-classification-lesson/master/colab_camera.py

Import libraries and define a callback function to handle (conerting format and drawing) images.

In [ ]:
import colab_camera
import IPython
from google.colab import output
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import base64

def run(img_str):
  #decode to image
  decimg = base64.b64decode(img_str.split(',')[1], validate=True)
  decimg = Image.open(BytesIO(decimg))
  decimg = np.array(decimg, dtype=np.uint8); 
  decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

  # image classify
  img = cv2.resize(decimg, (64, 64))
  img = img_to_array(img)
  img = img / 255.0
  img = img[np.newaxis]
  img = np.asarray(img)
  preds = keras_model.predict(img)
  result = np.argmax(preds[0])

  # draw output
  out_img = decimg
  cv2.putText(out_img, labels[result], (10,50), \
      cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)

  #encode to string
  _, encimg = cv2.imencode(".jpg", out_img,
      [int(cv2.IMWRITE_JPEG_QUALITY), 80])
  img_str = encimg.tobytes()
  img_str = ("data:image/jpeg;base64," +
      base64.b64encode(img_str).decode('utf-8'))
  return IPython.display.JSON({'img_str': img_str})

output.register_callback('notebook.run', run)

Showing the image of hand shape, "gu"(=Rock), "choki" (=Scissors) or "pa"(=Paper) on live camera.

In [ ]:
colab_camera.use_cam()

# Reference Link (in Japanese)

- https://qiita.com/a2kiti/items/f32de4f51a31d609e5a5